In [16]:
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime

In [17]:
df = pd.read_excel("marketing_data.xlsx")

In [18]:
df.dtypes

ID                       int64
Year_Birth               int64
Education               object
Marital_Status          object
Income                 float64
Kidhome                  int64
Teenhome                 int64
Dt_Customer             object
Recency                  int64
MntWines                 int64
MntFruits                int64
MntMeatProducts          int64
MntFishProducts          int64
MntSweetProducts         int64
MntGoldProds             int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumCatalogPurchases      int64
NumStorePurchases        int64
NumWebVisitsMonth        int64
Response                 int64
Complain                 int64
dtype: object

In [19]:
df.isnull().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
Response                0
Complain                0
dtype: int64

In [20]:
df.shape

(2240, 22)

In [21]:
# mean fill the null values 
mean_income = df['Income'].mean()
df['Income'] = df['Income'].fillna(mean_income)

In [22]:
#combine amount spent
df['MntSpent'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] 
+ df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']

0       518
1        44
2        47
3         0
4        45
       ... 
2235    173
2236     27
2237     34
2238    377
2239    332
Length: 2240, dtype: int64

In [23]:
#combine number of purchases without deals
df['NumPurchases'] = df['NumWebPurchases'] + df['NumCatalogPurchases'] + df['NumStorePurchases']

In [24]:
#Standardize Dt_Customer
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%m/%d/%y')

In [25]:
#birthyear should always occur before Dt_customer
mask = df['Year_Birth'] <= df['Dt_Customer'].dt.year
print(mask)
df = df[mask]

0       True
1       True
2       True
3       True
4       True
        ... 
2235    True
2236    True
2237    True
2238    True
2239    True
Length: 2240, dtype: bool


In [26]:
#convert birthyear into Age
current_year = datetime.now().year
df['Age'] = current_year - df['Year_Birth']

In [27]:
#convert Dt_Customer into numerical value
current_date = pd.Timestamp.now()
df['DaysEnrolled'] = (current_date - df['Dt_Customer']).dt.days

In [28]:
df.sample(20)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,Complain,MntSpent,NumPurchases,Age,DaysEnrolled
1126,10708,1978,2n Cycle,Single,36975.0,1,0,2013-12-28,50,10,...,1,0,3,7,0,0,27,4,46,3707
1437,6661,1974,Graduation,Married,67445.0,0,1,2012-12-08,63,757,...,9,6,12,6,0,0,1054,27,50,4092
334,3678,1973,Graduation,Widow,60208.0,1,1,2012-07-10,13,488,...,8,3,7,7,1,0,582,18,51,4243
480,10127,1965,Graduation,Married,58692.0,0,1,2014-06-04,21,301,...,8,2,4,7,0,0,373,14,59,3549
412,8962,1975,Master,Married,44319.0,1,1,2014-04-17,18,26,...,2,0,3,5,0,0,34,5,49,3597
1916,4998,1962,Graduation,Together,76081.0,0,0,2014-05-23,85,292,...,4,5,4,2,0,0,737,13,62,3561
1033,8825,1962,Graduation,Together,51195.0,1,1,2013-01-04,46,230,...,9,2,5,8,0,0,400,16,62,4065
1955,10703,1975,Master,Single,46098.0,1,1,2012-08-18,86,57,...,3,2,2,8,0,0,84,7,49,4204
474,4939,1946,Graduation,Together,37760.0,0,0,2012-08-31,20,84,...,4,1,6,7,0,0,127,11,78,4191
1645,4252,1965,Graduation,Married,44375.0,0,1,2014-03-23,73,213,...,4,1,6,5,0,0,246,11,59,3622


In [31]:
#Standardize Marital Status
df['Marital_Status'] = df['Marital_Status'].replace(['Alone', 'YOLO', 'Absurd'], 'Single')

In [34]:
df['Education'].value_counts()

Education
Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: count, dtype: int64

In [29]:
#Take features and target to use
df = df.drop(['ID', 'Dt_Customer', 'Year_Birth', 'Kidhome', 'Teenhome', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth'], axis=1)
df.dtypes

Education             object
Marital_Status        object
Income               float64
Recency                int64
NumDealsPurchases      int64
Response               int64
Complain               int64
MntSpent               int64
NumPurchases           int64
Age                    int64
DaysEnrolled           int64
dtype: object